In [1]:
import pandas as pd 
from numpy import int16

In [2]:
#DODAC ROZDIZAŁ NA ZLECENI Z PŁYTĄ PROSTOKĄTNĄ 1250x2500MM

In [20]:
#@title Funkcje
def generuj_tableke_na_cnc(_zlecenie):
  # zlecenie_ngl = _zlecenie.split("\n")[1].split("\t")[1:]
  zlecenie_ngl = ["MATERIAŁ",	"NUMER",	"ILOŚĆ",	"WYMIAR",	"OZNACZ.",	"CZAS"]
  zlecenie = list()
  for r in _zlecenie.split("\n")[2:]:
    r = r.split("\t")
    if len(r) == 7 and r[0] not in ["", "L.P."]:
      zlecenie.append(r[1:])

  df = pd.DataFrame(zlecenie, columns=zlecenie_ngl)

  df["GRUBOSC"] = df["MATERIAŁ"].apply(lambda x: x[-4:])


  def mat_typ(x):
    if "Sklejka" in x:
      return "SKL"
    elif "wiórowa" in x:
      return "PWI"

  def szer(x):
    new_str = ""
    x = x.lower()
    for c in x:
      if c not in "* ":
        new_str += c
    return new_str.strip().split("x")[0]

  def dlug(x):
    new_str = ""
    x = x.lower()
    for c in x:
      if c != "*" or c != " ":
        new_str += c
    return new_str.strip().split("x")[1]

  df["SZER"] = df["WYMIAR"].apply(szer).astype(int16)
  df["DLUG"] = df["WYMIAR"].apply(dlug).astype(int16)
  df["MAT_TYP"] = df["MATERIAŁ"].apply(mat_typ)
  df["ILOSC"] = df["ILOŚĆ"].astype(int16)

  return df

def drukuj_dane_cnc(df,MAT):

  _naglowek = "index$,material$,krok,sloje,grubosc,ilosc,mosty,nazwafmc$,kolor,fmcinclude$\n"
  content = list()
  content.append(_naglowek)
  # print(pwi_naglowek)
  for i in df[["NUMER", "ILOSC"]].values.tolist():
    if MAT == "S":
      a = [f"{i[0]}_SKL_X1_Msklejka,Sklejka,90,0,0,{i[1]},0,SKL",f"{i[0]}_SKL_X1_MSklejka.fmc,2,SKL","kontury"]
    elif MAT == "P":
      a = [f"{i[0]}_PWI_X1_MWIOR,WIOR,90,0,0,{i[1]},0,WIOR",f"{i[0]}_PWI_X1_MWIOR.fmc,2,WIOR","kontury"]
    b = chr(92).join(a)+chr(92)+"\n"
    # b = "/".join(a)+"/"+"\n"
    # print(b)
    content.append(b)

  content[-1] = content[-1].replace("\n", "")
  return content

def podziel_zlecenie(zlec):
  naglowki_zlecen = list(zlec.keys())

  for n in naglowki_zlecen:
    df = generuj_tableke_na_cnc(zlec[n])
    filtry = list(df.groupby(["MAT_TYP", "GRUBOSC"]).indices.keys())

    for f in filtry:
      if f[0] == "PWI":
        content = drukuj_dane_cnc(df[(df.MAT_TYP==f[0]) & (df.GRUBOSC == f[1])], "P")

      elif f[0] == "SKL":
        content = drukuj_dane_cnc(df[(df.MAT_TYP==f[0]) & (df.GRUBOSC == f[1]) & (df.DLUG <= 1525)], "S")

      fi = open(f"{n}_{f[0]}_{f[1].upper()}.csv", "w")
      # fi = open(f"{n}_{f[0]}.csv", "w")
      fi.writelines(content)
      fi.close()

def podziel_zlecenie_SKLP(zlec):
  naglowki_zlecen = list(zlec.keys())

  for n in naglowki_zlecen:
    df = generuj_tableke_na_cnc(zlec[n])
    filtry = list(df[(df.MAT_TYP=="SKL")].groupby(["MAT_TYP", "GRUBOSC"]).indices.keys())

    for f in filtry:

      content = drukuj_dane_cnc(df[(df.MAT_TYP=="SKL") & (df.GRUBOSC == f[1]) & (df.DLUG > 1525)], "S")

      fi = open(f"{n}_SKLP_{f[1].upper()}.csv", "w")
      # fi = open(f"{n}_{f[0]}.csv", "w")
      fi.writelines(content)
      fi.close()

In [21]:

#wzór na zlecenie
# """
# L.P.	MATERIAŁ	NUMER	ILOŚĆ	WYMIAR	OZNACZ.	CZAS
# 1	Płyta wiórowa 16mm	1278	40	*** 115x245	CN	20
# 2	Płyta wiórowa 16mm	1279	80	*** 110x222	CN	40
# 3	Płyta wiórowa 16mm	1393	40	*** 158x260	CN	20
_zlecenie = {

"AMA_2_1_24":
              """
L.P.	MATERIAŁ	NUMER	ILOŚĆ	WYMIAR	OZNACZ.	CZAS
1	Płyta wiórowa 16mm	1765	1040	*** 120x472	CN	3657:20 (211)
	RAZEM :		1040			3657:20



L.P.	MATERIAŁ	NUMER	ILOŚĆ	WYMIAR	OZNACZ.	CZAS
1	Sklejka 18mm	3003	230	*** 120x472	CN	808:50 (211)
2	Sklejka 18mm	3004	40	*** 140x757	CN	140:40 (211)
3	Sklejka 18mm	3005	40	*** 407x1070	CN	140:40 (211)
4	Sklejka 18mm	3006	40	*** 140x987	CN	140:40 (211)
5	Sklejka 18mm	3007	40	*** 450x1401	CN	140:40 (211)
6	Sklejka 18mm	3009	20	*** 397x1380	CN	70:20 (211)
7	Sklejka 18mm	3011	20	*** 411x1416	CN	70:20 (211)
8	Sklejka 18mm	3013	20	*** 457x1458	CN	70:20 (211)
9	Sklejka 18mm	3015	40	*** 415x1440	CN	140:40 (211)
10	Sklejka 18mm	3016	60	*** 357x1234	CN	211:00 (211)
11	Sklejka 18mm	3017	60	*** 140x765	CN	211:00 (211)
12	Sklejka 18mm	3018	30	*** 234x1635	CN	105:30 (211)
13	Sklejka 18mm	3019	30	*** 462X1216	CN	105:30 (211)
14	Sklejka 18mm	3020	30	*** 252x1653	CN	105:30 (211)
15	Sklejka 18mm	3021	30	*** 462x1234	CN	105:30 (211)
16	Sklejka 18mm	3022	30	*** 294x1677	CN	105:30 (211)
17	Sklejka 18mm	3023	30	*** 512x1300	CN	105:30 (211)
18	Sklejka 18mm	3024	60	*** 238x1660	CN	211:00 (211)
19	Sklejka 18mm	3025	60	*** 510x1280	CN	211:00 (211)
20	Sklejka 18mm	3026	50	*** 240x860	CN	175:50 (211)
21	Sklejka 18mm	3027	100	*** 240x842	CN	351:40 (211)
22	Sklejka 18mm	3029	50	*** 180x800	CN	175:50 (211)
23	Sklejka 18mm	3030	50	*** 180x782	CN	175:50 (211)
24	Sklejka 18mm	3032	20	*** 140x1860	CN	70:20 (211)
25	Sklejka 18mm	3039	405	*** 114x180	CN	1424:15 (211)
26	Sklejka 18mm	3041	90	*** 80x822	CN	316:30 (211)
27	Sklejka 18mm	3042	130	*** 80x822	CN	457:10 (211)
28	Sklejka 18mm	3043	10	*** 80x802	CN	35:10 (211)
29	Sklejka 18mm	3044	120	*** 114x632	CN	422:00 (211)
30	Sklejka 18mm	3045	420	*** 44x144	CN	1477:00 (211)
31	Sklejka 18mm	3048	70	*** 114x558	CN	246:10 (211)
32	Sklejka 18mm	3049	30	*** 80x1225	CN	105:30 (211)
33	Sklejka 18mm	3050	30	*** 80x1205	CN	105:30 (211)
34	Sklejka 18mm	3051	30	*** 80x1075	CN	105:30 (211)
35	Sklejka 18mm	3052	20	*** 80x840	CN	70:20 (211)
36	Sklejka 18mm	3053	20	*** 80x860	CN	70:20 (211)
37	Sklejka 18mm	3054	20	*** 80x880	CN	70:20 (211)
38	Sklejka 18mm	3055	60	*** 80x365	CN	211:00 (211)
39	Sklejka 18mm	3056	40	*** 80x840	CN	140:40 (211)
40	Sklejka 18mm	3057	30	*** 80x878	CN	105:30 (211)
41	Sklejka 18mm	3062	20	*** 80x700	CN	70:20 (211)
42	Sklejka 18mm	3064	120	*** 80x702	CN	422:00 (211)
43	Sklejka 18mm	3067	20	*** 140x2000	CN	70:20 (211)
44	Sklejka 18mm	3075	20	*** 140x1800	CN	70:20 (211)
45	Sklejka 18mm	3076	20	*** 140x1600	CN	70:20 (211)
46	Sklejka 18mm	3077	20	*** 140x1400	CN	70:20 (211)
47	Sklejka 18mm	3087	30	*** 140x900	CN	105:30 (211)
48	Sklejka 18mm	3106	10	*** 114x632	CN	35:10 (211)
49	Sklejka 18mm	3108	15	*** 240x930	CN	52:45 (211)
50	Sklejka 18mm	3109	30	*** 240x912	CN	105:30 (211)
51	Sklejka 18mm	3110	15	*** 180x870	CN	52:45 (211)
52	Sklejka 18mm	3111	15	*** 180x852	CN	52:45 (211)
53	Sklejka 18mm	3125	50	*** 80x800	CN	175:50 (211)
54	Sklejka 18mm	3130	60	*** 140x1542	CN	211:00 (211)
55	Sklejka 18mm	3131	60	*** 188x889	CN	211:00 (211)
56	Sklejka 18mm	3133	20	*** 114x944	CN	70:20 (211)
	RAZEM :		3230			11358:50


 """

}


In [22]:
podziel_zlecenie(_zlecenie)

In [23]:
podziel_zlecenie_SKLP(_zlecenie)